In [1]:
# !ls /root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/images/val

In [2]:
img_name = '000000006251.jpg'
data_split = 'train'

# img_name = '000000003126.jpg'
# data_split = 'val'

# img_name = '000000042188.jpg'
# data_split = 'test'
camera_positions = ['A_Pillar_Driver', 'Rear_Mirror', 'A_Pillar_Codriver']
camera_position = camera_positions[2]
img_path = f'/root/data/processed/synthetic_cabin_bw/{camera_position}/images/{data_split}/{img_name}'
# img_path = './tmp_img/drive_and_act.jpg'
print(img_path)

/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/images/train/000000006251.jpg


In [3]:
from src.modules.human_detector.human_detector import HumanDetector

detector = HumanDetector(
    config_path='./src/modules/human_detector/config/faster_rcnn.py',
    pretrained_path="https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/"\
        "faster_rcnn_r101_caffe_fpn_1x_coco/faster_rcnn_r101_caffe_fpn_1x_coco_bbox_mAP-0.398_20200504_180057-b269e9dd.pth",
    checkpoint_path="mmengine_workdir/human_detector/epoch_1.pth",
    data_root_path='/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/',
    device='cuda:0',
    working_directory='./mmengine_workdir/human_detector',
    log_level='CRITICAL'
    # log_level='INFO'
)

detector.load_pretrained()
# detector.update_config()
detector_result = detector.get_bbox(img_path)
bbox = detector_result['bboxes']
print(bbox)

self.load_from_checkpoint True


wandb: Currently logged in as: gentlerainsky. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmdet.visualization.local_visualizer.DetLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


tensor([[ 591.0479,  309.5541,  600.2931,  310.2175],
        [1054.2123,  173.9538, 1061.0823,  174.7231],
        [1273.4204,  246.8016, 1277.7184,  247.4343],
        [1275.8143,  245.8495, 1278.6068,  246.4850],
        [ 825.5535,  297.0514,  835.3415,  297.6357],
        [ 594.8972,  308.8106,  602.2758,  309.6338],
        [ 451.3365,  315.4344,  460.0471,  316.2794],
        [ 816.5244,  303.1110,  827.6404,  303.6942],
        [1227.4291,  268.4538, 1237.6071,  269.0270],
        [ 899.6608,  224.2229,  907.2375,  224.7556],
        [1277.8625,  227.3450, 1279.4651,  227.8831],
        [ 456.7484,  316.0577,  465.4175,  316.9548],
        [ 566.9140,  283.5357,  570.9199,  284.2993],
        [ 823.5587,  299.5082,  833.2197,  300.0787],
        [ 829.9935,  301.9643,  840.6600,  302.5647],
        [ 734.9769,  374.9871,  790.2085,  376.4621],
        [ 778.2689,  298.5451,  789.6831,  299.0808],
        [ 892.5544,  226.9589,  901.6031,  227.3225],
        [ 755.8926,  296.156

In [ ]:
from src.modules.pose_estimator_2d.pose_estimator_2d import PoseEstimator2D
import numpy as np

pose_estimator_2d = PoseEstimator2D(
    config_path='src/modules/pose_estimator_2d/config/hrnet.py',
    pretrained_path='https://download.openmmlab.com/mmpose/v1' \
        '/body_2d_keypoint/topdown_heatmap/coco'\
            '/td-hm_hrnet-w32_8xb64-210e_coco-256x192-81c58e40_20220909.pth',
    # checkpoint_path="mmengine_workdir/pose_estimator_2d/best_coco_AP_epoch_0.pth",
    checkpoint_path="mmengine_workdir/pose_estimator_2d/best_coco_AP_epoch_9.pth",
    data_root_path='/root/data/processed/synthetic_cabin_bw/A_Pillar_Codriver/',
    device='cuda:0',
    working_directory='mmengine_workdir/pose_estimator_2d',
    log_level='CRITICAL'
)

pose_estimator_2d.load_pretrained()
# pose_estimator_2d_result = pose_estimator_2d.inference(img_path, bbox.detach().cpu().numpy(), bbox_format='xywh')
pose_estimator_2d_result = pose_estimator_2d.inference(img_path, np.array([[0, 0, 1200, 1000]]), bbox_format='xywh')
keypoints_2D = pose_estimator_2d_result[0].pred_instances['keypoints'][0]
print(keypoints_2D)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

keypoints = pd.DataFrame(keypoints_2D, columns=['x', 'y'])

img = mpimg.imread(img_path)
imgplot = plt.imshow(img)
plt.scatter(keypoints['x'], keypoints['y'])
plt.show()

In [ ]:
import json
import numpy as np
with open(f'/root/data/processed/synthetic_cabin_bw/{camera_position}/annotations/person_keypoints_{data_split}.json') as f:
    data = json.loads(f.readline())

annotations = {}
for id in range(len(data['images'])):
    annotations[data['images'][id]['file_name']] = {
        'keypoints': np.array(data['annotations'][id]['keypoints']).reshape(-1, 3)[:, :2]
    }

In [ ]:
pose_estimator_2d.visualize(
    img_path=img_path,
    bboxes=bbox.tolist(),
    bbox_format='xywh',
    gt_keypoints=np.expand_dims(annotations[img_name]['keypoints'], 0)
)